In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4e2d71401e58df0eb28ef94a9fde4ad46bb2316b8dac09e63b61f6b2e36dd706
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence_transformers


In [2]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

from sentence_transformers import SentenceTransformer, util
import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/arxiv-cs-papers-abstract-from-2010/cs_arxiv_from_2010.csv


In [3]:
docs_df = pd.read_csv('/kaggle/input/arxiv-cs-papers-abstract-from-2010/cs_arxiv_from_2010.csv')
docs_df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,authors,title,category,abstract
0,704.0213,Ketan D. Mulmuley Hariharan Narayanan,Geometric Complexity Theory V: On deciding non...,['cs.CC'],This article has been withdrawn because it h...
1,704.1409,Yao HengShuai,Preconditioned Temporal Difference Learning,"['cs.LG', 'cs.AI']",This paper has been withdrawn by the author....
2,704.1829,"Stefan Felsner, Kamil Kloch, Grzegorz Matecki,...",On-line Chain Partitions of Up-growing Semi-or...,['cs.DM'],On-line chain partition is a two-player game...
3,705.0561,Jing-Chao Chen,Iterative Rounding for the Closest String Problem,"['cs.DS', 'cs.CC']",The closest string problem is an NP-hard pro...
4,705.1025,David Eppstein,Recognizing Partial Cubes in Quadratic Time,['cs.DS'],We show how to test whether a graph with n v...


In [4]:
device = 'cuda'
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [5]:
docs_text = (docs_df['title'] + ' ' + docs_df['abstract']).values.tolist()
docs_text[:5]

["Geometric Complexity Theory V: On deciding nonvanishing of a generalized\n  Littlewood-Richardson coefficient   This article has been withdrawn because it has been merged with the earlier\narticle GCT3 (arXiv: CS/0501076 [cs.CC]) in the series. The merged article is\nnow available as:\n  Geometric Complexity Theory III: on deciding nonvanishing of a\nLittlewood-Richardson Coefficient, Journal of Algebraic Combinatorics, vol. 36,\nissue 1, 2012, pp. 103-110. (Authors: Ketan Mulmuley, Hari Narayanan and Milind\nSohoni)\n  The new article in this GCT5 slot in the series is:\n  Geometric Complexity Theory V: Equivalence between blackbox derandomization\nof polynomial identity testing and derandomization of Noether's Normalization\nLemma, in the Proceedings of FOCS 2012 (abstract), arXiv:1209.5993 [cs.CC]\n(full version) (Author: Ketan Mulmuley)\n",
 'Preconditioned Temporal Difference Learning   This paper has been withdrawn by the author. This draft is withdrawn for its\npoor quality in

In [6]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]
        text = ' '.join(text)
        
    return text

In [7]:
def clean_data(data):
    cleaned_data = []
    for doc in notebook.tqdm(data):
        text = clean_text(doc, False)
        cleaned_data.append(text)
    return cleaned_data

In [8]:
cleaned_docs = clean_data(docs_text)

  0%|          | 0/484027 [00:00<?, ?it/s]

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [10]:
def get_embeddings(data):
    return embedder.encode(data, convert_to_tensor=True)

In [11]:
abstract_embeddings = get_embeddings(cleaned_docs)

Batches:   0%|          | 0/15126 [00:00<?, ?it/s]

In [12]:
query = ['temporal expression extraction']
query_embedding = get_embeddings(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
cos_scores = util.cos_sim(query_embedding, abstract_embeddings)
cos_scores.shape

torch.Size([1, 484027])

In [18]:
cos_scores[0]

tensor([ 0.0448,  0.3572,  0.0271,  ...,  0.1939, -0.0275,  0.0822],
       device='cuda:0')

In [19]:
top_results = torch.topk(cos_scores[0], k=5)

In [20]:
top_results

torch.return_types.topk(
values=tensor([0.7779, 0.6950, 0.6800, 0.6714, 0.6708], device='cuda:0'),
indices=tensor([ 16069, 154036,  66896,  13600, 192765], device='cuda:0'))

In [25]:
top_results.indices.detach()

tensor([ 16069, 154036,  66896,  13600, 192765], device='cuda:0')

Let's see the closest match

In [23]:
docs_df.iloc[16069].abstract

'  Automatic annotation of temporal expressions is a research challenge of great\ninterest in the field of information extraction. In this report, I describe a\nnovel rule-based architecture, built on top of a pre-existing system, which is\nable to normalise temporal expressions detected in English texts. Gold standard\ntemporally-annotated resources are limited in size and this makes research\ndifficult. The proposed system outperforms the state-of-the-art systems with\nrespect to TempEval-2 Shared Task (value attribute) and achieves substantially\nbetter results with respect to the pre-existing system on top of which it has\nbeen developed. I will also introduce a new free corpus consisting of 2822\nunique annotated temporal expressions. Both the corpus and the system are\nfreely available on-line.\n'